# Image Labeler
This example shows you how to use Syd to create an interactive image labeler. 

Suppose you have a stack of images (for example, a stack of imaging ROIs from 2P imaging or histology) and you want to label them with different categories. You can use Syd to create an image labeler to label the images interactively.

Here, we just show how to do it with randomly colored letters for simplicity, but you could use this as a scaffold for any kind of data!

In [1]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    pass

This is just to create a dataset of images to label...

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import string
import io
from PIL import Image

plt.close('all')

# List of typical named colors
named_colors = [
    "red",
    "green",
    "blue",
    "orange",
    "purple",
]

# Settings for image size
width, height = 100, 100

def figure_to_numpy(fig, dpi=100):
    """Render a Matplotlib figure to a NumPy RGB array."""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=dpi, bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    arr = np.array(image)
    buf.close()
    return arr

def render_letter_image(letter, color, width=width, height=height):
    fig, ax = plt.subplots(figsize=(width / 100, height / 100), dpi=100)
    ax.text(0.5, 0.5, letter, fontsize=60, ha='center', va='center', color=color)
    ax.set_axis_off()
    fig.subplots_adjust(0, 0, 1, 1)
    arr = figure_to_numpy(fig)
    plt.close(fig)
    return arr

# Generate the image stack
letter_colors = [random.choice(named_colors) for _ in string.ascii_uppercase]
letter_images = [render_letter_image(letter, color) for letter, color in zip(string.ascii_uppercase, letter_colors)]

In [ ]:
from syd import Viewer

# We'll use the colors as our "labels"... if you are labeling ROIs you might call change this to something more biological like:
# ['neuron', 'astrocyte', 'unlabeled']... or ['tdtomato', 'gfp', 'both', 'none']...
labels = named_colors

# The annotations is a dictionary that will store any labels or names you make
annotations = {"descriptions": {}, "label": {}}

# We'll initiate it just for a demonstration
annotations["label"][0] = letter_colors[0]
annotations["descriptions"][0] = "A" # the first uppercase letter is 'A'

class ImageLabeler(Viewer):
    def __init__(self, images: np.ndarray, labels: list[str], annotations: dict = {}):
        self.images = images
        self.labels = labels
        self._not_labeled_str = "not labeled"
        self.annotations = annotations
        labels = [self._not_labeled_str] + self.labels

        self.add_integer("image", value=0, min=0, max=len(self.images)-1)
        self.add_selection("type", options=labels)
        self.add_text("description")

        self.on_change("image", self.reset_annotations)
        self.on_change("type", self.annotate_type)
        self.on_change("description", self.annotate_description)
        self.reset_annotations(self.state)

    def annotate_type(self, state):
        self.annotations["label"][state["image"]] = state["type"]
    
    def annotate_description(self, state):
        self.annotations["descriptions"][state["image"]] = state["description"]
    
    def get_annotations(self, state):
        current_label = annotations["label"].get(state["image"], None)
        current_description = annotations["descriptions"].get(state["image"], None)
        return current_label, current_description

    def reset_annotations(self, state):
        current_label, current_description = self.get_annotations(state)
        self.update_selection("type", value=current_label or self._not_labeled_str)
        self.update_text("description", value=current_description or "")

    def plot(self, state):
        fig, ax = plt.subplots(1, 1, figsize=(4, 4))
        ax.imshow(self.images[state["image"]])

        title = f"Image {state['image']}"
        current_label, current_description = self.get_annotations(state)                
        if current_label is not None:
            title += f"\nLabel: {current_label}"
        if current_description is not None:
            title += f"\nDescription: {current_description}"
        ax.set_title(title)
        return fig

viewer = ImageLabeler(letter_images, labels, annotations)
viewer.show()

In [ ]:
# Then something to save your annotations...
np.save("annotations.npy", viewer.annotations, allow_pickle=True)